**CURSO**: Análisis Geoespacial, Departamento de Geociencias y Medio Ambiente, Universidad Nacional de Colombia - sede Medellín <br/>
**Profesor**: Edier Aristizábal (evaristizabalg@unal.edu.co) <br />
**Classroom code**: [32cjlau] <br />
**Credits**: The content of this notebook is taken from several sources, such as: [open-geo-tutorial](https://github.com/ceholden/open-geo-tutorial), [geohackweek](https://geohackweek.github.io/raster/). Every effort has been made to trace copyright holders of the materials used in this book. The author apologies for any unintentional omissions and would be pleased to add an acknowledgment in future editions. 

# Importar datos raster

## Archivos ASCII

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
geologia=np.loadtxt(r'G:\My Drive\CATEDRA\MACHINE LEARNING\datos\raster/gamma.asc', skiprows=6)
geologia=np.where(geologia==-9999.,np.nan,geologia)
plt.imshow(geologia)
plt.colorbar();

## Archivos TXT

In [ ]:
FS_hazard  = np.genfromtxt(r'G:\My Drive\INVESTIGACION\PAPERS\ELABORACION\SHIA_George\resultados/PR_AMEACA.txt')
FS_hazard = np.where(FS_hazard==-9999.,np.nan,FS_hazard)
plt.imshow(FS_hazard)
plt.colorbar();

## Geospatial Libraries

**Gdal**: [GDAL](https://gdal.org/) (Geospatial Data Abstraction Library) is a translator library for raster and vector geospatial data formats.

In [ ]:
from osgeo import gdal

**Rasterio**:  [Rasterio](https://rasterio.readthedocs.io/en/latest/) is a Geographic information systems use GeoTIFF and other formats to organize and store gridded raster datasets such as satellite imagery and terrain models. Rasterio reads and writes these formats and provides a Python API based on Numpy N-dimensional arrays and GeoJSON.

In [ ]:
import rasterio as rio

In [ ]:
#importar una imagen (banda)
data = gdal.Open("https://landsat-pds.s3.amazonaws.com/c1/L8/009/056/LC08_L1TP_009056_20210228_20210228_01_RT/LC08_L1TP_009056_20210228_20210228_01_RT_B3.TIF")

In [ ]:
#para obtener el gdal.band
banda = data.GetRasterBand(1)

#para convertirlo en array
b4 = banda.ReadAsArray()

#para graficar uan matriz se utiliza la funcion imshow
import matplotlib.pyplot as plt
plt.imshow(b4)
plt.colorbar()

In [ ]:
#importar uan imagen compuesta
composite = gdal.Open(r'G:\My Drive\CATEDRA\SENSORES REMOTOS\Imagen\barranquilla\Composite_LE70090532003066EDC00.tif')

#para saber el número de bandas
print(composite.RasterCount)

## Sat_search

In [ ]:
!pip install sat-search

In [ ]:
import satsearch
from satsearch import Search

In [ ]:
bbox = [ 11.756057739257812,
          57.649809962218995,
          12.10693359375,
          57.751442372568924
       ]

url = 'https://earth-search.aws.element84.com/v0'
bbox_search = Search(
    bbox=bbox,
    datetime="2020-10-01/2021-02-01",
    query={"eo:cloud_cover": {"lt": 1}},
    collections=["sentinel-s2-l2a-cogs"],
    url=url,
)
items = bbox_search.items()
print(items.summary())

In [ ]:
from IPython.display import JSON
JSON(items[0].assets)

In [ ]:
# Read and open(B4 and B8)
b4 = rio.open(items[0].asset("red")["href"])
red = b4.read()

In [ ]:
from rasterio.plot import show
fig, ax = plt.subplots(figsize=(8,6))
show(red,cmap="viridis", ax=ax)

In [ ]:
# NOTE this STAC API endpoint does not currently search the entire catalog

bbox = (-124.71, 45.47, -116.78, 48.93) #(west, south, east, north)

timeRange = '2019-01-01/2020-10-01'

# STAC metadata properties
properties =  ['eo:row=027',
               'eo:column=047',
               'landsat:tier=T1']

results = Search.search(collection='landsat-8-l1',
                        bbox=bbox,
                        datetime=timeRange,
                        property=properties,
                        sort=['<datetime'],
                        url=url,
                        )

print('%s items' % results.found())
items = results.items()
items.save('subset.geojson')

In [ ]:
import geopandas as gpd
gf = gpd.read_file("subset.geojson")
gf.head()

In [ ]:
from ipywidgets import interact
from IPython.display import display, Image

def browse_images(items):
    n = len(items)

    def view_image(i=0):
        item = items[i]
        print(f"id={item.id}\tdate={item.datetime}\tcloud%={item['eo:cloud_cover']}")
        display(Image(item.asset('thumbnail')['href']))

    interact(view_image, i=(0,n-1))

In [ ]:
browse_images(items)

In [ ]:
polygon = {
    "type": "Polygon",
    "coordinates": [
        [
            [11.756057739257812, 57.649809962218995],
            [12.10693359375, 57.649809962218995],
            [12.10693359375, 57.751442372568924],
            [11.756057739257812, 57.751442372568924],
            [11.756057739257812, 57.649809962218995],
        ]
    ],
}

intersect_search = Search(intersects=polygon, url=url)
print('Query returned {} items'.format(bbox_search.found()))

## Sentinel

In [ ]:
import os
from shapely.geometry import MultiPolygon, Polygon

In [ ]:
import sentinelsat
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
from datetime import date

In [ ]:
user = 'edieraristizabal' 
file = open("Clave Copernicus.txt", "r")
for line in file:
    password = file.readlines()

api = SentinelAPI(user, password[2], 'https://scihub.copernicus.eu/dhus')

In [ ]:
footprint = geojson_to_wkt(read_geojson('map.geojson')) # el poligono debe ser sencillo, si tiene mucho vertice como un mapa genera error

In [ ]:
aoi="POINT(6.15 -75.31)"

In [ ]:
products = api.query(aoi, date=('20210101', '20210228'), platformname = 'Sentinel-2', cloudcoverpercentage = (0,20))

In [ ]:
len(products)

In [ ]:
products_gdf = api.to_dataframe(products)
products_gdf_sorted = products_gdf.sort_values(['cloudcoverpercentage'], ascending=[True])
products_gdf_sorted.head(3)

In [ ]:
api.download("b6986e18-1324-4fb6-a2e0-0658da90a3d0")

In [ ]:
!unzip S2B_MSIL2A_20190220T080929_N0211_R049_T32CMB_20190220T115112.zip --quite

In [ ]:
from zipfile import ZipFile
ZipFile("S2B_MSIL2A_20191009T073939_N0213_R063_T32CMB_20191009T130037.zip").extractall()

In [ ]:
!gdalinfo S2B_MSIL2A_20191009T073939_N0213_R063_T32CMB_20191009T130037.zip

In [ ]:
!gdalinfo S2B_MSIL2A_20191009T073939_N0213_R063_T32CMB_20191009T130037.SAFE\MTD_MSIL2A.xml

Tener en cuenta la primera parte del archivo SENTINEL2_L2A, la ultima parte puede cambiar de acuerdo con el nivel de procesamiento de la imagen 'L1C', etc. Al igual que el CRS en términos de EPSG, el cual puede ser obtenido de la función anterior.

In [ ]:
!gdalinfo S2B_MSIL2A_20191009T073939_N0213_R063_T32CMB_20191009T130037.SAFE\MTD_MSIL2A.xml:10m:EPSG_32732

In [ ]:
!gdal_translate SENTINEL2_L2A:S2B_MSIL2A_20191009T073939_N0213_R063_T32CMB_20191009T130037.SAFE\MTD_MSIL2A.xml:10m:EPSG_32732 \
                 10m.tif \
                 -co TILED=YES --config GDAL_CACHEMAX 1000 --config GDAL_NUM_THREADS 2

In [ ]:
dataset = gdal.Open('10m.tif')
print(dataset)

In [ ]:
num_bands = dataset.RasterCount
print('Number of bands in image: {n}\n'.format(n=num_bands))

In [ ]:
b4 = dataset.GetRasterBand(4)
b4 = b4.ReadAsArray()
print(b4)

In [ ]:
b4=np.where(b4==0,np.nan,b4)
fig, ax = plt.subplots(1, figsize=(10, 10))
plt.imshow(b4)
plt.colorbar();


## AWS

Landsat 8 data is available for anyone to use via Amazon S3. All Landsat 8 scenes are available from the start of imagery capture. All new Landsat 8 scenes are made available each day, often within hours of production.

The Landsat program is a joint effort of the U.S. Geological Survey and NASA. First launched in 1972, the Landsat series of satellites has produced the longest, continuous record of Earth’s land surface as seen from space. NASA is in charge of developing remote-sensing instruments and spacecraft, launching the satellites, and validating their performance. USGS develops the associated ground systems, then takes ownership and operates the satellites, as well as managing data reception, archiving, and distribution. Since late 2008, Landsat data have been made available to all users free of charge. Carefully calibrated Landsat imagery provides the U.S. and the world with a long-term, consistent inventory of vitally important global resources.

AWS has made Landsat 8 data freely available on Amazon S3 so that anyone can use our on-demand computing resources to perform analysis and create new products without needing to worry about the cost of storing Landsat data or the time required to download it.

### Accessing Landsat on [AWS](https://registry.opendata.aws/landsat-8/)
The data are organized using a directory structure based on each scene’s path and row. For instance, the files for Landsat scene LC08_L1TP_139045_20170304_20170316_01_T1 are available in the following location: 

s3://landsat-pds/c1/L8/139/045/LC08_L1TP_139045_20170304_20170316_01_T1/

The “c1” refers to Collection 1, the “L8” refers to Landsat 8, “139” refers to the scene’s path, “045” refers to the scene’s row, and the final directory matches the product’s identifier, which uses the following naming convention: LXSS_LLLL_PPPRRR_YYYYMMDD_yyymmdd_CC_TX, in which:

- L = Landsat
- X = Sensor
- SS = Satellite
- PPP = WRS path
- RRR = WRS row
- YYYYMMDD = Acquisition date
- yyyymmdd = Processing date
- CC = Collection number
- TX = Collection category

In this case, the scene corresponds to WRS path 139, WRS row 045, and was taken on March 4th, 2017.

Each scene’s directory includes:

- a .TIF GeoTIFF for each of the scene’s up to 12 bands (note that the GeoTIFFs include 512x512 internal tiling)
.TIF.ovr overview file for each .TIF (useful in GDAL based applications)
- a _MTL.txt metadata file
- a small rgb preview jpeg, 3 percent of the original size
- a larger rgb preview jpeg, 15 percent of the original size
- an index.html file that can be viewed in a browser to see the RGB preview and links to the GeoTIFFs and metadata files

For instance, the files associated with scene LO08_L1TP_009056_20201108_20201120_01_T1 are available at:

s3://landsat-pds/c1/L8/009/056/LC08_L1TP_009056_20201108_20201120_01_T1/

or

https://landsat-pds.s3.amazonaws.com/c1/L8/009/056/LO08_L1TP_009056_20201108_20201120_01_T1/index.html

A gzipped csv describing all available scenes is available at

s3://landsat-pds/scene_list.gz

or

https://landsat-pds.s3.amazonaws.com/c1/L8/scene_list.gz



https://aws.amazon.com/blogs/aws/start-using-landsat-on-aws/

In [ ]:
path = 'https://landsat-pds.s3.amazonaws.com/c1/L8/009/056/LC08_L1TP_009056_20210228_20210228_01_RT/LC08_L1TP_009056_20210228_20210228_01_RT_B1.TIF'
dataset = gdal.Open(path)
b2 = dataset.GetRasterBand(1)
b2 = b2.ReadAsArray()
print(b2)

In [ ]:
print(dataset.RasterCount)

In [ ]:
b2=np.where(b2==0,np.nan,b2)
fig, ax = plt.subplots(1, figsize=(10, 10))
plt.imshow(b2)
plt.colorbar();

## Google Cloud

[Cloud Storage](https://cloud.google.com/storage/docs) allows world-wide storage and retrieval of any amount of data at any time. You can use Cloud Storage for a range of scenarios including serving website content, storing data for archival and disaster recovery, or distributing large data objects to users via direct download.

Cloud Storage provides a variety of public datasets that can be accessed by the community and integrated into their applications. Google pays for the hosting of these datasets, providing public access to the data via the Google Cloud Console, gsutil, or with the Cloud Storage API.

### Available public datasets on Cloud Storage
- [Landsat](https://console.cloud.google.com/storage/browser/gcp-public-data-landsat?pageState=(%22StorageObjectListTable%22:(%22f%22:%22%255B%255D%22))&prefix=&forceOnObjectsSortingFiltering=false): A satellite image dataset from the United States Geological Survey (USGS) and NASA that includes millions of multispectral images of the Earth's land surface, at resolutions of between 15 and 60 meters per pixel, from 1982 through the present.

- [Sentinel-2](https://console.cloud.google.com/storage/browser/gcp-public-data-sentinel-2;tab=objects?_ga=2.243846213.1981017282.1614808297-1548957655.1604964455&prefix=&forceOnObjectsSortingFiltering=false): A satellite image dataset from the European Space Agency (ESA) that includes multispectral images of the Earth's land surface, with a resolution of 10–60 meters per pixel, from 2015 through the present.

- [NEXRAD](https://console.cloud.google.com/storage/browser/gcp-public-data-nexrad-l2;tab=objects?_ga=2.244393029.1981017282.1614808297-1548957655.1604964455&prefix=&forceOnObjectsSortingFiltering=false): A weather radar dataset collected from a network of 160 high-resolution Doppler weather radars operated by the NOAA National Weather Service (NWS), the Federal Aviation Administration (FAA), and the U.S. Air Force (USAF).

In [ ]:
url = "https://storage.googleapis.com/gcp-public-data-landsat/LC08/01/042/034/LC08_L1TP_042034_20170616_20170629_01_T1/LC08_L1TP_042034_20170616_20170629_01_T1_B4.TIF"

l8 = gdal.Open(url)
b4 = l8.GetRasterBand(1)
b4 = b4.ReadAsArray()

In [ ]:
b4=np.where(b4==0,np.nan,b4)
fig, ax = plt.subplots(1, figsize=(10, 10))
plt.imshow(b4)
plt.colorbar();

### Additional sources

- [EarthAI](https://astraea.earth/platform/)